<a href="https://colab.research.google.com/github/lee4205/Potato_Chip_Classification/blob/master/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd Potato_Chip_Classification

In [ ]:
import sys
import os
import csv
import subprocess
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

In [ ]:
cwd = os.getcwd()
subprocess.run(["mkdir " + cwd + "/dataset"], shell=True)

In [ ]:
image_width = 1280 // 10
image_height = 960 // 10
image_size = (image_width, image_height)
image_channel = 1

In [ ]:
flavors = os.listdir(cwd + "/potato-chips")
with open("potato_chips.csv", 'a') as c:
    csv_input = csv.writer(c)
    pixels = []
    pixel = 0
    for x in range((image_width)*(image_height)):
        pixel += 1
        pixels.append('pixel'+ str(pixel))
    header = ["image", "flavor"]
    header.extend(pixels)
    csv_input.writerow(header)
    for flavor in flavors:
        images = os.listdir(cwd + f"/potato-chips/{flavor}")
        print("loading image from " + flavor + " ...")
        for image in images:
            rgb_image = Image.open(cwd + f"/potato-chips/{flavor}/" + image)
            grey_image = rgb_image.convert('L').resize((image_width, image_height))
            pixel_data = np.asarray(grey_image.getdata(), dtype=np.int).reshape((grey_image.size[1], grey_image.size[0]))
            pixel_data = pixel_data.flatten()
            image_data = [image, flavors.index(flavor)]
            image_data.extend(pixel_data)
            csv_input.writerow(image_data)
        subprocess.run(["cp " + cwd + "/potato-chips/" + flavor + "/*.jpg " + cwd + "/dataset"], shell=True)

In [ ]:
df = pd.read_csv(cwd + "/potato_chips.csv")
train_df, validate_df = train_test_split(df, train_size=0.6, random_state=42)
validate_df, test_df = train_test_split(validate_df, test_size=0.5, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [ ]:
# デバッグ用
# train_df
# validate_df
# test_df
# plt.figure(figsize = (15,15))
# sns.set_style("darkgrid")
# sns.countplot(train_df['flavor'])
# sns.countplot(validate_df['flavor'])
# sns.countplot(test_df['flavor'])

In [ ]:
y_train = train_df['flavor']
y_validate = validate_df['flavor']
y_test = test_df['flavor']
y = test_df['flavor']
del train_df['image'], train_df['flavor']
del validate_df['image'], validate_df['flavor']
del test_df['image'], test_df['flavor']

In [ ]:
label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)
y_validate = label_binarizer.fit_transform(y_validate)
y_test = label_binarizer.fit_transform(y_test)

In [ ]:
x_train = train_df.values
x_validate = validate_df.values
x_test = test_df.values

In [ ]:
x_train = x_train / 255
x_validate = x_validate / 255
x_test = x_test / 255

In [ ]:
x_train = x_train.reshape(-1,image_height,image_width,1)
x_validate = x_validate.reshape(-1,image_height,image_width,1)
x_test = x_test.reshape(-1,image_height,image_width,1)

In [ ]:
# デバッグ用
f, ax = plt.subplots(3,3) 
f.set_size_inches(10, 10)
k = 0
for i in range(3):
    for j in range(3):
        ax[i,j].imshow(x_train[k].reshape(image_height,image_width) , cmap = "gray")
        k += 1
    plt.tight_layout()    

In [ ]:
datagen = ImageDataGenerator(featurewise_center=False, # set input mean to 0 over the dataset
                             samplewise_center=False, # set each sample mean to 0
                             featurewise_std_normalization=False, # divide inputs by std of the dataset
                             samplewise_std_normalization=False, # divide each input by its std
                             zca_whitening=False, # apply ZCA whitening
                             rotation_range=10, # randomly rotate images in the range (degrees, 0 to 180)
                             zoom_range = 0.1, # Randomly zoom image 
                             width_shift_range=0.1, # randomly shift images horizontally (fraction of total width)
                             height_shift_range=0.1, # randomly shift images vertically (fraction of total height)
                             horizontal_flip=False, # randomly flip images
                             vertical_flip=False) # randomly flip images
datagen.fit(x_train)

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                            patience =2,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

In [ ]:
model = Sequential()
model.add(Conv2D(75 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (image_width,image_height,image_channel)))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(50 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(25 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Flatten())
model.add(Dense(units = 512 , activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(units = 7 , activation = 'softmax'))
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.summary()

In [ ]:
history = model.fit(datagen.flow(x_train,y_train, batch_size = 5),
                    epochs = 200,
                    validation_data = (x_validate, y_validate),
                    callbacks = [learning_rate_reduction])

In [ ]:
print("Accuracy of the model is - " , model.evaluate(x_validate,y_validate)[1]*100 , "%")

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(15,5))
axs[0].plot(range(1,len(history.history["accuracy"])+1),history.history["accuracy"])
axs[0].plot(range(1,len(history.history["val_accuracy"])+1),history.history["val_accuracy"])
axs[0].set_title("Model Accuracy")
axs[0].set_ylabel("Accuracy")
axs[0].set_xlabel("Epoch")
axs[0].set_xticks(np.arange(1,len(history.history["accuracy"])+1),len(history.history["accuracy"])/10)
axs[0].legend(["train", "val"], loc="best")
axs[1].plot(range(1,len(history.history["loss"])+1),history.history["loss"])
axs[1].plot(range(1,len(history.history["val_loss"])+1),history.history["val_loss"])
axs[1].set_title("Model Loss")
axs[1].set_ylabel("Loss")
axs[1].set_xlabel("Epoch")
axs[1].set_xticks(np.arange(1,len(history.history["loss"])+1),len(history.history["loss"])/10)
axs[1].legend(["train", "val"], loc="best")
plt.show()

In [ ]:
predictions = model.predict_classes(x_test)
for i in range(len(predictions)):
    if(predictions[i] >= 9):
        predictions[i] += 1
predictions[:5] 

In [ ]:
classes = ["Class " + str(i) for i in range(8) if i != 0]
print(classification_report(y, predictions, target_names = classes))

In [ ]:
cm = confusion_matrix(y,predictions)
cm = pd.DataFrame(cm , index = [i for i in range(8) if i != 0] , columns = [i for i in range(8) if i != 0])
plt.figure(figsize = (10,10))
sns.heatmap(cm,cmap= "Blues", linecolor = 'black' , linewidth = 1 , annot = True, fmt='')

In [ ]:
correct = (predictions == y).to_numpy().nonzero()[0]

In [ ]:
i = 0
plt.figure(figsize = (10,10))
for c in correct[:9]:
    plt.subplot(3,3,i+1)
    plt.imshow(x_test[c].reshape(image_height,image_width), cmap="gray", interpolation='none')
    plt.title("Predicted Class {}, Actual Class {}".format(predictions[c], y[c]))
    plt.tight_layout()
    i += 1